# Global vs. Local Variables

Local Variable Priority: This is called "function scope"

In [2]:
x = 5
def f(y,z):
    x = 3
    result = x + y + z
    return result

print(f(1, 2))

6


Unless the "global" keyword is used

In [5]:
x = 5
def f(y,z):
    global x
    result = x + y + z
    x = 3
    return result

print(f(1, 2))

8


In [6]:
x = 5
def f(y,z):
    result = x + y + z
    x = 3
    return result

print(f(1, 2))

UnboundLocalError: local variable 'x' referenced before assignment

Global Variables are used automatically without error if there aren't any locals to use

In [3]:
x = 5
def f(y,z):
    result = x + y + z
    return result

print(f(1, 2))

8


The return keyword ends a function immediately; "return" with nothing after is equivalent to "return None"

In [9]:
def f():
    print("Start")
    return
    print("End")            # dead code

f()

Start


A function with no return statement will perform all of the actions defined (side effects) and then returns None

In [10]:
def f(x):
    result = x + 10

print(f(10))

None


Print is not the same as return

In [20]:
def _double(x):
    print(2 * x)

_double(3)
print("----")
print(_double(3))
# print(2 * double(3)) # is broken

6
----
6
None


In [4]:
def double(x):
    return 2 * x

double(3)
print(double(3))
print("----")
print(2 * double(3))


6
----
12


Composing functions

In [25]:
def f(x):
    return 3 * x

def g(x):
    return x + 2

def h(x):
    return f(g(x))

print(h(1))

9


In [23]:
def f(x):
    return 10 * x

def g(y,z):
    return f(3*y) + z

def h(a):
    return f(g(a,f(a+1)))

print(h(1))

500


## Higher Order Functions

In Python, functions are treated as "first-class objects" meaning they are treated just like numbers and strings. Functions are a way of abstracting from a procedure in order to adhere to the principle of being lazy. Programming with functions is all about finding ways to reduce code repetition.

How would we apply the function "double" twice?

In [6]:
print(double(double(3)))

12


The first way to generalize our code would be to capture a specific composition of functions.

In [7]:
def double_twice(x):
    return double(double(x))

Since functions are just objects, we can pass them as arguments.

In [10]:
def do_twice(function, x):
    return function(function(x))

do_twice(double, 3)

12

We can generalize even more with an anonymous function.

In [14]:
def do_twice_factory(function):
    return lambda x: function(function(x))

do_twice_factory(double)(4)

16

Examples of higher order functions

In [23]:
def summation(bot, top, f, nx):
    summed = 0
    x = bot
    while x <= top:
        summed += f(x)
        x = nx(x)
    return summed

def sum_squares(bot, top):
    return summation(bot, top, lambda x: x**2, lambda x: x+1)

def approx_pi(bot, top):
    return (8 * summation(bot, top, lambda x: 1.0/x**2, lambda x: x+2)) ** 0.5

approx_pi(1, 1000000)

3.1415920169700393

In [46]:
import math
def numeric_integration(f, a, b):
    dx = 0.000001
    return dx * summation(a, b, f, lambda x: x + dx)

def derivative(f):
    dx = 0.0001
    return lambda x: (f(x+dx)-f(x))/dx

print(numeric_integration(lambda x: math.sin(x), 0, 1))
print(derivative(lambda x: x**2)(5))

0.4596972733957041
10.000099999984968


One really useful function is the fixed point function. It repeatedly applies a specific function to an initial point until the value hits an equilibrium.

In [38]:
def epsilon(e, a, b):
    return abs(a-b)<e

def fixed_point(f, guess):
    nx = f(guess)
    while not epsilon(0.0001, guess, nx):
        guess = nx
        nx = f(nx)
    return nx

def sqrt(x):
    return fixed_point(lambda y: (y + x/y)/2, 1.0)

sqrt(2)

1.4142135623746899

(Lexical) Closures

In [37]:
def add(x):
    def add_x(y):
        return x + y
    return add_x

print(add(1))
print(add(1)(2))
add_3 = add(3)
print(add_3(2))

<function add.<locals>.add_x at 0x7f4e6ae9b620>
3
5


Functions are objects; they can be stored anywhere objects can be stored

In [42]:
funcs = [type, add(3), double, float]
for f in funcs:
    print(f(1))

<class 'int'>
4
2
1.0


Functions can be defined pretty much anywhere

In [55]:
fun = []
for i in range(10):
    def mul_i(x, i=i):
        return i * x
    fun.append(mul_i)

print([f(2) for f in fun])

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Why "i=i"? This has to do with Python doing something called late binding. Late Binding means that Python looks up a function call while it is running instead of before it runs. The "i=i" forces Python to do early binding instead because default arguments need to be looked up early.

In [56]:
fun = []
for i in range(10):
    def mul_i(x):
        return i * x
    fun.append(mul_i)

print([f(2) for f in fun])

[18, 18, 18, 18, 18, 18, 18, 18, 18, 18]


Python stores its runtime variables in a dictionary that you can actually access called "locals" and "globals" with the difference being that "locals" calls inside a certain "scope" will only give the local environment.

In [61]:
print(locals().get('add')(3)(2))
print(globals().get('add')(1)(2))

def g(x,y):
    z = 3
    print(locals())
    
g(1,2)

5
3
{'z': 3, 'y': 2, 'x': 1}
